# EECS759P Coursework 1
- Name: Bheki Maenetja
- Student ID: 230382466

## Imports

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, deque
import math
import hashlib
import string
import random
random.seed(123)
from statistics import mean, stdev
from itertools import permutations as perm

## Agenda-Based Search

### Loading Data

In [2]:
# Function provided in undirected_map.py
def load_data(df):
    station_dict = defaultdict(list)
    zone_dict = defaultdict(dict)

    # get data row by row
    for index, row in df.iterrows():
        start_station = row[0]
        end_station = row[1]

        line = row[2] # slight modification; adding line of station to each node.
        
        act_cost = int(row[3])
        
        zone1 = row[4]
        zone2 = row[5]

        # station dictionary of child station tuples (child_name, line, cost from parent to the child)
        # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
        station_list = station_dict[start_station]
        station_list.append((end_station, line, act_cost))

        # the following two lines add the other direction of the tube "step"
        station_list = station_dict[end_station]
        station_list.append((start_station, line, act_cost))

        # we add the main zone
        zone_dict[start_station]["main"] = zone1
        # we add the secondary zone

        if zone2 != "0":
            zone_dict[start_station]["2nd"] = zone2
            # if the secondary zone is not 0 it's the main zone for the ending station
            zone_dict[end_station]["main"] = zone2
        else:
            # otherwise the main zone for the ending station is the same as for the starting station
            zone_dict[end_station]["main"] = zone1

    return station_dict, zone_dict

In [3]:
tube_df = pd.read_csv("tubedata.csv", header=None)
tube_df.head()

,0,1,2,3,4,5
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0


In [4]:
stations, zones = load_data(tube_df)

### Node Class

In [5]:
class Node:
    def __init__(self, data, parent=None):
        self.name = data[0]
        self.line = data[1]
        self.cost = data[2]
        self.parent = parent

    def __str__(self):
        return f"{self.name} via {self.line} (t = {self.cost})"

    def __repr__(self):
        return self.__str__()

### Building Optimal Path to Goal Node

In [6]:
def build_path(node, print_results=True):
    """
    Builds path found by given search function
    using goal node.
    """
    if node is None:
        print("NO PATH FOUND")
        return [], 0
    
    path = [node]
    path_cost = node.cost
    
    while node.parent:
        node = node.parent
        path = [node] + path
        path_cost = path_cost if node.cost is None else path_cost + node.cost

    if print_results:
        path_str = f"{path[0].name} -> " + " -> ".join([n.__str__() for n in path[1:]])
        print(f"PATH: {path_str}")
        print(f"\nPath cost (avg. travel time): {path_cost} minutes")
    
    return path, path_cost

### Depth-First Search

In [7]:
def dfs_search(start, goal, print_steps=True):
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    # Create initial node for start state.
    start_node = Node((start, None, None), None)

    # Expand start node and add its children to frontier.
    frontier = [
        Node(s, start_node) 
        for s in stations[start].copy()
    ]
    
    explored = [start]
    num_explored = 1 # we've "explored" the start node by looking at it
    num_expanded = 1 # we obviously expand the start node to begin the search
    
    if print_steps: print(f"Start: {start} —>")
    
    while frontier:
        node = frontier.pop() # take node from the end; LIFO
        num_explored += 1 # increment explored count

        if print_steps: print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            if print_steps:
                print("Success!!!")
                print(f"GOAL: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
            return node, num_explored, num_expanded
        
        # Node expansion
        num_expanded += 1 # increment expansion count
        new_nodes = stations[node.name].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

    if print_steps:
        print("Failure.")
        print(f"Final Node: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
    return None, num_explored, num_expanded

In [8]:
goal_node, _, _ = dfs_search("Euston", "Victoria")

Start: Euston —>
Warren Street via Victoria (t = 1) ->
Oxford Circus via Victoria (t = 2) ->
Green Park via Victoria (t = 2) ->
Victoria via Victoria (t = 2) ->
Success!!!
GOAL: Victoria | Nodes explored = 5 | Nodes expanded = 4


In [9]:
_, _ = build_path(goal_node)

PATH: Euston -> Warren Street via Victoria (t = 1) -> Oxford Circus via Victoria (t = 2) -> Green Park via Victoria (t = 2) -> Victoria via Victoria (t = 2)

Path cost (avg. travel time): 7 minutes


### Breadth-First Search

In [10]:
def bfs_search(start, goal, print_steps=True):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    # Create initial node for start state.
    start_node = Node((start, None, None), None)

    # Expand start node and add its children to frontier.
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]
    
    explored = [start]
    num_explored = 1 # we've "explored" the start node by looking at it
    num_expanded = 1 # we obviously expand the start node to begin the search
    
    if print_steps: print(f"Start: {start} —>")
    
    while frontier:
        node = frontier.pop(0) # take node from the start; FIFO
        num_explored += 1 # increment explored count

        if print_steps: print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            if print_steps:
                print("Success!!!")
                print(f"GOAL: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
            return node, num_explored, num_expanded

        # Node expansion
        num_expanded += 1 # increment expanded count
        new_nodes = stations[node.name].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

    if print_steps:
        print("Failure.")
        print(f"Final Node: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
    return None, num_explored, num_expanded

In [11]:
goal_node, _, _ = bfs_search("Euston", "Victoria")

Start: Euston —>
Mornington Crescent via Northern (t = 2) ->
Warren Street via Northern (t = 1) ->
King's Cross St. Pancras via Northern (t = 2) ->
King's Cross St. Pancras via Victoria (t = 2) ->
Warren Street via Victoria (t = 1) ->
Camden Town via Northern (t = 1) ->
Goodge Street via Northern (t = 2) ->
Oxford Circus via Victoria (t = 2) ->
Euston Square via Circle (t = 2) ->
Farringdon via Circle (t = 4) ->
Angel via Northern (t = 2) ->
Caledonian Road via Piccadilly (t = 5) ->
Russell Square via Piccadilly (t = 2) ->
Highbury & Islington via Victoria (t = 4) ->
Kentish Town via Northern (t = 2) ->
Chalk Farm via Northern (t = 2) ->
Mornington Crescent via Northern (t = 1) ->
Warren Street via Northern (t = 2) ->
Tottenham Court Road via Northern (t = 1) ->
Regent's Park via Bakerloo (t = 2) ->
Piccadilly Circus via Bakerloo (t = 2) ->
Bond Street via Central (t = 1) ->
Green Park via Victoria (t = 2) ->
Great Portland Street via Circle (t = 2) ->
King's Cross St. Pancras via Circ

In [12]:
_, _ = build_path(goal_node)

PATH: Euston -> Warren Street via Northern (t = 1) -> Oxford Circus via Victoria (t = 2) -> Green Park via Victoria (t = 2) -> Victoria via Victoria (t = 2)

Path cost (avg. travel time): 7 minutes


### Uniform Cost Search

In [13]:
def ucs_search(start, goal, print_steps=True):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    # Create initial node for start state.
    start_node = Node((start, None, None), None)

    # Expand start node and add its children to frontier.
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]

    # Sort nodes in frontier by cost.
    frontier.sort(key=lambda x: x.cost)
    
    explored = [start]
    num_explored = 1 # we've "explored" the start node by looking at it
    num_expanded = 1 # we obviously expand the start node to begin the search

    if print_steps: print(f"START: {start} —>")
    
    while frontier:
        node = frontier.pop(0) # take node from start; FIFO
        num_explored += 1 # increment explored count

        if print_steps: print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            if print_steps:
                print("Success!!!")
                print(f"GOAL: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
            return node, num_explored, num_expanded
        
        # Node expansion
        num_expanded += 1 # increment expanded count
        new_nodes = stations[node.name].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

        # sort frontier again after adding new children
        frontier.sort(key=lambda x: x.cost)

    if print_steps:
        print("Failure.")
        print(f"Final Node: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
    return None, num_explored, num_expanded

In [14]:
goal_node, _, _ = ucs_search("Euston", "Victoria")

START: Euston —>
Warren Street via Northern (t = 1) ->
Warren Street via Victoria (t = 1) ->
Mornington Crescent via Northern (t = 2) ->
Camden Town via Northern (t = 1) ->
Mornington Crescent via Northern (t = 1) ->
King's Cross St. Pancras via Northern (t = 2) ->
King's Cross St. Pancras via Victoria (t = 2) ->
Goodge Street via Northern (t = 2) ->
Tottenham Court Road via Northern (t = 1) ->
Leicester Square via Northern (t = 1) ->
Covent Garden via Piccadilly (t = 1) ->
Oxford Circus via Victoria (t = 2) ->
Bond Street via Central (t = 1) ->
Marble Arch via Central (t = 1) ->
Kentish Town via Northern (t = 2) ->
Chalk Farm via Northern (t = 2) ->
Euston Square via Circle (t = 2) ->
Angel via Northern (t = 2) ->
Russell Square via Piccadilly (t = 2) ->
Warren Street via Northern (t = 2) ->
Holborn via Central (t = 2) ->
Chancery Lane via Central (t = 1) ->
Charing Cross via Northern (t = 2) ->
Embankment via Bakerloo (t = 1) ->
Piccadilly Circus via Piccadilly (t = 2) ->
Regent's Pa

In [15]:
_, _ = build_path(goal_node)

PATH: Euston -> Warren Street via Northern (t = 1) -> Oxford Circus via Victoria (t = 2) -> Green Park via Victoria (t = 2) -> Victoria via Victoria (t = 2)

Path cost (avg. travel time): 7 minutes


### Comparison of DFS, BFS and UCS

In [16]:
routes = [
    "Euston to Victoria",
    "Canada Water to Stratford",
    "New Cross Gate to Stepney Green",
    "Ealing Broadway to South Kensington",
    "Baker Street to Wembley Park",
    "Whitechapel to Westminster",
    "Rickmansworth to North Greenwich",
    "King's Cross St. Pancras to Upminster",
    "Colliers Wood to Debden",
    "Paddington to Holborn",
]

dfs_explored = []
dfs_expanded = []
dfs_path_cost = []

bfs_explored = []
bfs_expanded = []
bfs_path_cost = []

ucs_explored = []
ucs_expanded = []
ucs_path_cost = []


for r in routes:
    r_list = r.split(" to ")

    # DFS
    dfs_node, dfs_ex, dfs_ep = dfs_search(r_list[0], r_list[1], False)
    _, dfs_pc = build_path(dfs_node, False)
    dfs_explored.append(dfs_ex)
    dfs_expanded.append(dfs_ep)
    dfs_path_cost.append(dfs_pc)

    # BFS
    bfs_node, bfs_ex, bfs_ep = bfs_search(r_list[0], r_list[1], False)
    _, bfs_pc = build_path(bfs_node, False)
    bfs_explored.append(bfs_ex)
    bfs_expanded.append(bfs_ep)
    bfs_path_cost.append(bfs_pc)

    # UCS
    ucs_node, ucs_ex, ucs_ep = ucs_search(r_list[0], r_list[1], False)
    _, ucs_pc = build_path(ucs_node, False)
    ucs_explored.append(ucs_ex)
    ucs_expanded.append(ucs_ep)
    ucs_path_cost.append(ucs_pc)

data = {
    "Routes": routes,
    "DFS Nodes Explored": dfs_explored,
    "BFS Nodes Explored": bfs_explored,
    "UCS Nodes Explored": ucs_explored,
    "DFS Nodes Expanded": dfs_expanded,
    "BFS Nodes Expanded": bfs_expanded,
    "UCS Nodes Expanded": ucs_expanded,
    "DFS Path Cost (avg. travel time)": dfs_path_cost,
    "BFS Path Cost (avg. travel time)": bfs_path_cost,
    "UCS Path Cost (avg. travel time)": ucs_path_cost,
}

table = pd.DataFrame(data=data)
table

,Routes,DFS Nodes Explored,BFS Nodes Explored,UCS Nodes Explored,DFS Nodes Expanded,BFS Nodes Expanded,UCS Nodes Expanded,DFS Path Cost (avg. travel time),BFS Path Cost (avg. travel time),UCS Path Cost (avg. travel time)
0,Euston to Victoria,5,40,45,4,39,44,7,7,7
1,Canada Water to Stratford,6,44,204,5,43,203,15,15,14
2,New Cross Gate to Stepney Green,33,27,63,32,26,62,27,14,14
3,Ealing Broadway to South Kensington,179,52,101,178,51,100,57,20,33
4,Baker Street to Wembley Park,3,21,178,2,20,177,13,13,54
5,Whitechapel to Westminster,214,39,31,213,38,30,49,12,15
6,Rickmansworth to North Greenwich,172,196,175,171,195,174,116,60,84
7,King's Cross St. Pancras to Upminster,88,279,273,87,278,272,82,52,52
8,Colliers Wood to Debden,69,222,221,68,221,220,84,53,53
9,Paddington to Holborn,99,64,49,98,63,48,114,14,16


In [17]:
table.describe()

,DFS Nodes Explored,BFS Nodes Explored,UCS Nodes Explored,DFS Nodes Expanded,BFS Nodes Expanded,UCS Nodes Expanded,DFS Path Cost (avg. travel time),BFS Path Cost (avg. travel time),UCS Path Cost (avg. travel time)
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,86.800000,98.400000,134.000000,85.800000,97.400000,133.000000,56.400000,26.000000,34.200000
std,78.558966,95.302093,86.469005,78.558966,95.302093,86.469005,41.156072,20.363366,25.385035
min,3.000000,21.000000,31.000000,2.000000,20.000000,30.000000,7.000000,7.000000,7.000000
25%,12.750000,39.250000,52.500000,11.750000,38.250000,51.500000,18.000000,13.250000,14.250000
50%,78.500000,48.000000,138.000000,77.500000,47.000000,137.000000,53.000000,14.500000,24.500000
75%,153.750000,163.000000,197.500000,152.750000,162.000000,196.500000,83.500000,44.000000,52.750000
max,214.000000,279.000000,273.000000,213.000000,278.000000,272.000000,116.000000,60.000000,84.000000


### Extending the Cost Function

In [18]:
def ucs_search_2(start, goal, penalty=2, print_steps=True):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    # Create initial node for start state.
    start_node = Node((start, None, None), None)

    # Expand start node and add its children to frontier.
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]

    # Sort nodes in frontier by cost.
    frontier.sort(key=lambda x: x.cost)
    
    explored = [start]
    num_explored = 1 # we've "explored" the start node by looking at it
    num_expanded = 1 # we obviously expand the start node to begin the search

    if print_steps: print(f"START: {start} —>")
    
    while frontier:
        node = frontier.pop(0) # take node from start; FIFO
        num_explored += 1 # increment explored count

        if print_steps: print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            if print_steps:
                print("Success!!!")
                print(f"GOAL: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
            return node, num_explored, num_expanded

        
        # Node expansion
        num_expanded += 1 # increment expanded count
        new_nodes = [
            (c[0], c[1], c[2])
            if c[1] == node.line
            else (c[0], c[1], c[2] + penalty) # applying penalty for line change
            for c in stations[node.name].copy()
        ]
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

        # sort frontier again after adding new children
        frontier.sort(key=lambda x: x.cost)

    if print_steps:
        print("Failure.")
        print(f"Final Node: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
    return None, num_explored, num_expanded

In [19]:
goal_node, _, _ = ucs_search_2("Euston", "Victoria")

START: Euston —>
Warren Street via Northern (t = 1) ->
Warren Street via Victoria (t = 1) ->
Mornington Crescent via Northern (t = 2) ->
Camden Town via Northern (t = 1) ->
Mornington Crescent via Northern (t = 1) ->
King's Cross St. Pancras via Northern (t = 2) ->
King's Cross St. Pancras via Victoria (t = 2) ->
Goodge Street via Northern (t = 2) ->
Tottenham Court Road via Northern (t = 1) ->
Leicester Square via Northern (t = 1) ->
Kentish Town via Northern (t = 2) ->
Chalk Farm via Northern (t = 2) ->
Angel via Northern (t = 2) ->
Warren Street via Northern (t = 2) ->
Charing Cross via Northern (t = 2) ->
Tufnell Park via Northern (t = 2) ->
Belsize Park via Northern (t = 2) ->
King's Cross St. Pancras via Northern (t = 2) ->
Archway via Northern (t = 2) ->
Hampstead via Northern (t = 2) ->
Covent Garden via Piccadilly (t = 3) ->
Old Street via Northern (t = 3) ->
Moorgate via Northern (t = 1) ->
Embankment via Bakerloo (t = 3) ->
Waterloo via Bakerloo (t = 2) ->
Lambeth North via 

In [20]:
_, _ = build_path(goal_node)

PATH: Euston -> Warren Street via Northern (t = 1) -> Oxford Circus via Victoria (t = 4) -> Green Park via Victoria (t = 2) -> Victoria via Victoria (t = 2)

Path cost (avg. travel time): 9 minutes


### Heuristic Search

#### Heuristic Function

In [21]:
def get_lines(station):
    """
    Returns the set of all lines running
    through the given station.
    """
    return {s[1] for s in stations[station]}

def heuristic(current, goal):
    if current.name == goal:
        return 0

    utility = 1
    lines = get_lines(goal)
    zone_maps = {
        "1": 1,
        "2": 2,
        "3": 3,
        "4": 4,
        "5": 5,
        "6": 6,
        "a": 7,
        "b": 8,
        "c": 9,
        "d": 10,
    }

    current_zone = zone_maps[zones[current.name]["main"]]
    goal_zone = zone_maps[zones[goal]["main"]]

    zone_dist = abs(current_zone - goal_zone)
    utility = utility + zone_dist + int(not current.line in lines)

    return utility

#### Best-First Search

In [22]:
def best_first_search(start, goal, print_steps=True):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    # Create initial node for start state.
    start_node = Node((start, None, None), None)

    # Expand start node and add its children to frontier.
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]

    # Sort nodes in frontier by heuristic function.
    frontier.sort(key=lambda x: heuristic(x, goal))
    
    explored = [start]
    num_explored = 1 # we've "explored" the start node by looking at it
    num_expanded = 1 # we obviously expand the start node to begin the search

    if print_steps: print(f"START: {start} —>")
    
    while frontier:
        node = frontier.pop(0) # take node from start; FIFO
        num_explored += 1 # increment explored count

        if print_steps: print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            if print_steps:
                print("Success!!!")
                print(f"GOAL: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
            return node, num_explored, num_expanded
        
        # Node expansion
        num_expanded += 1 # increment expanded count
        new_nodes = stations[node.name].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

        # sort frontier again according to heuristic function
        frontier.sort(key=lambda x: heuristic(x, goal))

    if print_steps:
        print("Failure.")
        print(f"Final Node: {node.name} | Nodes explored = {num_explored} | Nodes expanded = {num_expanded}")
    return None, num_explored, num_expanded

In [23]:
goal_node, _, _ = best_first_search("Euston", "Victoria")

START: Euston —>
King's Cross St. Pancras via Victoria (t = 2) ->
Warren Street via Victoria (t = 1) ->
Euston Square via Circle (t = 2) ->
Farringdon via Circle (t = 4) ->
Oxford Circus via Victoria (t = 2) ->
Great Portland Street via Circle (t = 2) ->
King's Cross St. Pancras via Circle (t = 2) ->
Barbican via Circle (t = 1) ->
Warren Street via Victoria (t = 2) ->
Green Park via Victoria (t = 2) ->
Victoria via Victoria (t = 2) ->
Success!!!
GOAL: Victoria | Nodes explored = 12 | Nodes expanded = 11


In [24]:
_, _ = build_path(goal_node)

PATH: Euston -> Warren Street via Victoria (t = 1) -> Oxford Circus via Victoria (t = 2) -> Green Park via Victoria (t = 2) -> Victoria via Victoria (t = 2)

Path cost (avg. travel time): 7 minutes


#### Comparison with Original UCS

In [25]:
routes = [
    "Euston to Victoria",
    "Canada Water to Stratford",
    "New Cross Gate to Stepney Green",
    "Ealing Broadway to South Kensington",
    "Baker Street to Wembley Park",
    "Whitechapel to Westminster",
    "Rickmansworth to North Greenwich",
    "King's Cross St. Pancras to Upminster",
    "Colliers Wood to Debden",
    "Paddington to Holborn",
]

ucs_explored = []
ucs_expanded = []
ucs_path_cost = []

hs_explored = []
hs_expanded = []
hs_path_cost = []

for r in routes:
    r_list = r.split(" to ")

    # Heuristic Search (Best-First Search)
    hs_node, hs_ex, hs_ep = best_first_search(r_list[0], r_list[1], False)
    _, hs_pc = build_path(hs_node, False)
    hs_explored.append(hs_ex)
    hs_expanded.append(hs_ep)
    hs_path_cost.append(hs_pc)

    # UCS
    ucs_node, ucs_ex, ucs_ep = ucs_search(r_list[0], r_list[1], False)
    _, ucs_pc = build_path(ucs_node, False)
    ucs_explored.append(ucs_ex)
    ucs_expanded.append(ucs_ep)
    ucs_path_cost.append(ucs_pc)

data = {
    "Routes": routes,
    "UCS Nodes Explored": ucs_explored,
    "Best First Search Nodes Explored": hs_explored,
    "UCS Nodes Expanded": ucs_expanded,
    "Best First Search Nodes Expanded": hs_expanded,
    "UCS Path Cost (avg. travel time)": ucs_path_cost,
    "Best First Search Path Cost (avg. travel time)": hs_path_cost,
}

table = pd.DataFrame(data=data)
table

,Routes,UCS Nodes Explored,Best First Search Nodes Explored,UCS Nodes Expanded,Best First Search Nodes Expanded,UCS Path Cost (avg. travel time),Best First Search Path Cost (avg. travel time)
0,Euston to Victoria,45,12,44,11,7,7
1,Canada Water to Stratford,204,7,203,6,14,15
2,New Cross Gate to Stepney Green,63,13,62,12,14,14
3,Ealing Broadway to South Kensington,101,12,100,11,33,20
4,Baker Street to Wembley Park,178,4,177,3,54,13
5,Whitechapel to Westminster,31,12,30,11,15,15
6,Rickmansworth to North Greenwich,175,31,174,30,84,60
7,King's Cross St. Pancras to Upminster,273,204,272,203,52,52
8,Colliers Wood to Debden,221,47,220,46,53,53
9,Paddington to Holborn,49,23,48,22,16,14


In [26]:
table.describe()

,UCS Nodes Explored,Best First Search Nodes Explored,UCS Nodes Expanded,Best First Search Nodes Expanded,UCS Path Cost (avg. travel time),Best First Search Path Cost (avg. travel time)
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,134.000000,36.500000,133.000000,35.500000,34.200000,26.300000
std,86.469005,60.238876,86.469005,60.238876,25.385035,20.155231
min,31.000000,4.000000,30.000000,3.000000,7.000000,7.000000
25%,52.500000,12.000000,51.500000,11.000000,14.250000,14.000000
50%,138.000000,12.500000,137.000000,11.500000,24.500000,15.000000
75%,197.500000,29.000000,196.500000,28.000000,52.750000,44.000000
max,273.000000,204.000000,272.000000,203.000000,84.000000,60.000000


## Adversarial Search

### Genetic Algorithm Fitness Function (from password_fitness.py)

In [27]:
MY_USERNAME = "ecs23496"

def get_password(student_username, l=10):
    # Possible characters include upper-case English letters, numbers between 0 and 9 (inclusive), 
    # and the underscore symbol
    options = string.digits + string.ascii_uppercase  + "_"

    h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
      s += options[n%len(options)]

    return s[0:l]

# TO DO: replace *** with your EECS username and uncomment the code
student_password = get_password(MY_USERNAME)
print(student_password)

# Distance function
def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        # Square of the absolute difference between two Unicode codes
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score


# Upper bound of the distance value
MAX_VALUE = distance_function('0000000000', '__________')

# Compute normalised fitness for a list of candidate passwords 
def get_normalised_fitness(list_of_phrases, student_password):
    ordered_dict = dict()
    phrase_to_find = student_password
    for phrase in list_of_phrases:
        # Return 1 when a candidate matches the true password (string distance between them is zero)
        ordered_dict[phrase] = 1 - distance_function(phrase, phrase_to_find) / MAX_VALUE
    return ordered_dict

# Example of how to get fitness values for a list of candidates
get_normalised_fitness(['2Q4HHHHOTJ', '2HHZQYUOTJ'], student_password)

ZS09_JG55Z


{'2Q4HHHHOTJ': 0.48234584062986685, '2HHZQYUOTJ': 0.32453125283957984}

### Genetic Algorithm Implementation

In [28]:
def initialise_population(pop_size):
    options = string.digits + string.ascii_uppercase  + "_"
    return ["".join(random.sample(options, 10)) for _ in range(pop_size)]

def assign_fitness(population, target):
    return get_normalised_fitness(population, target)

def cross_over(cross_pool, cross_prob=0.5):
    """
    Performs 1-point crossover of indivduals. 
    """
    
    offspring = []
    other_parents = cross_pool.copy()
    
    for parent in cross_pool:
        rand_prob = random.random()
        if rand_prob <= cross_prob and parent in other_parents and len(other_parents) > 1:
            other_parents.remove(parent)
            other_parent = random.choice(other_parents)
            other_parents.remove(other_parent)
            cross_point = random.randint(0,10)

            child_1 = parent[:cross_point] + other_parent[cross_point:]
            child_2 = other_parent[:cross_point] + parent[cross_point:]

            offspring.append(child_1)
            offspring.append(child_2)

    return offspring

def mutation(mut_pool, mut_prob=0.5):
    mut_children = []
    
    for child in mut_pool:
        child = list(child)
        for i, letter in enumerate(child):
            rand_prob = random.random()
            if rand_prob <= mut_prob:
                randIdx = random.randint(0, len(child) - 1)
                temp = child[randIdx]
                child[randIdx] = child[i]
                child[i] = temp

        mut_children.append("".join(child))

    return mut_children

def genetic_algorithm(target, n_gen=10000, pop_size=1000, cross_prob=0.5, mut_prob=0.5, print_results=True):
    half = pop_size // 2
    generations = n_gen
    
    # Initialise population
    new_pop = initialise_population(pop_size)

    if any(w == target for w in new_pop):
        print("Got it!!!")
        return target, 0
    
    # Assign fitness
    fitness = assign_fitness(new_pop, target)

    # Mating Selection
    new_pop.sort(key=lambda x: fitness[x], reverse=True)

    if print_results:
        print(f"Fittest initial individual: {new_pop[0]} (f={fitness[new_pop[0]]})")
    
    for i in range(n_gen): # n_gen is the ceiling for the number of reproductions.
        mating_pool = new_pop.copy()[:half]
        
        # Crossover
        mating_pool = cross_over(mating_pool, cross_prob)

        # Mutation
        mating_pool = mutation(mating_pool, mut_prob)

        # Selection
        new_pop = new_pop.copy()[:half] + mating_pool
        fitness = assign_fitness(new_pop, target)
        new_pop.sort(key=lambda x: fitness[x], reverse=True)

        if fitness[new_pop[0]] == 1:
            generations = i + 1
            break

    if print_results:
        print(f"Fittest final individual: {new_pop[0]} (f={fitness[new_pop[0]]}); found after {generations} generations.")
        print(f"Actual target: {target}")

    return target, generations

In [29]:
_, _, = genetic_algorithm(student_password)

Fittest initial individual: OMB8_0D67V (f=0.6753914775331477)
Fittest final individual: ZS09_JG55Z (f=1.0); found after 571 generations.
Actual target: ZS09_JG55Z


### Number of Reproductions

In [30]:
# Find the average number of reproductions needed to converge on solution with default hyperparameters.
N = 10
repros = []

for i in range(N):
    _, num_gen = genetic_algorithm(student_password, print_results=False)
    repros.append(num_gen)

data = {
    "Iteration": [f"Iteration {i+1}" for i in range(N)],
    "Number of Reproductions": repros
}

table = pd.DataFrame(data=data)
table

,Iteration,Number of Reproductions
0,Iteration 1,756
1,Iteration 2,689
2,Iteration 3,773
3,Iteration 4,666
4,Iteration 5,595
5,Iteration 6,439
6,Iteration 7,718
7,Iteration 8,877
8,Iteration 9,326
9,Iteration 10,626


In [31]:
table.describe()

,Number of Reproductions
count,10.000000
mean,646.500000
std,162.368066
min,326.000000
25%,602.750000
50%,677.500000
75%,746.500000
max,877.000000


### Hyperparameters

In [32]:
# Find the average number of reproductions needed to converge on solution with different
# combinations of crossover probability and mutation probability.

repro_means = []
repro_stds = []

perms = list(perm([0.25, 0.5, 0.75], 2)) # take different combinations of crossover and mutation probability

for cross_prob, mut_prob in perms:
    gens = []
    
    for j in range(3):
        _, num_gen = genetic_algorithm(
            student_password, 
            cross_prob=cross_prob, 
            mut_prob=mut_prob, 
            print_results=False
        )
        
        gens.append(num_gen)
    
    repro_means.append(mean(gens))
    repro_stds.append(stdev(gens))

data = {
    "Crossover Probability": [p[0] for p in perms],
    "Mutation Probability": [p[1] for p in perms],
    "Avg. # of Reproductions": repro_means,
    "Standard Deviation": repro_stds,
}

table = pd.DataFrame(data=data)
table

,Crossover Probability,Mutation Probability,Avg. # of Reproductions,Standard Deviation
0,0.25,0.50,1192.333333,187.513555
1,0.25,0.75,8772.666667,2125.803691
2,0.50,0.25,87.666667,3.055050
3,0.50,0.75,8600.666667,1625.265927
4,0.75,0.25,77.666667,10.598742
5,0.75,0.50,622.333333,78.799323
